# Outline
The experiments section consists of two parts:
- PART1: Demonstrating the performance of our implementations of various intrinsic rewards;
- PART2: Discussion of various issues of the application of intrinsic rewards.

# Preliminary work

- Mingqi:
  - Update the rllte framework to adapt the latest reward class; [DONE]
  - Prepare the paper framework and write some preliminary contents. [DONE]
- Roger:
  - Finish the rest of the transfer work; [DONE]
  - Implement the *Disagree* reward module; [DONE]
  - Check the correctness of the workflow of the implemented modules.

- I have available training results of PPO on 57 atari games, so we don't need to train them again.
- Use Super Mario 1-1 without extrinsic rewards to test correctness

# Baseline Setting
- env: SuperMarioBros-1-1-v3
- image input preprocessing: x / 255.
- reward normalization: rms
- weight init: orthogonal
- reward filter: False
- update proportion: 1.0
- lstm: no

| Name     | RND      | ICM      | Pseudocounts | RIDE   | E3B      | RE3 | NGU   | Disagreement |
|----------|----------|----------|--------------|--------|----------|-----|-------|--------------|
| Done     | True     | True     | True         | True   | True     | True| False | True         |


# PART1
Using the baseline settings, report the performance for all intrinsic rewards 

--> Fill this table as we run more experiments

|Intrinsic Reward  | Obs Normalization | Reward Norm. | Weight Init. | Update Proportion | Memory Required |
|------------------|-------------------|--------------|--------------|-------------------|-----------------|
| Pseudocounts     |T                  |RMS           |orth.         |0.5                |F                |
| ICM              |T                  |None          |orth.         |1.0                |F                |
| RND              |T                  |None          |orth.         |0.1                |F                |
| E3B              |F                  |RMS           |orth.         |0.5                |F                |
| RIDE             |T                  |RMS           |orth.         |0.1                |F                |
| RE3              |F                  |MINMAX        |orth.         |1.0                |F                |
| NGU              |T                  |RMS           |orth.         |0.5                |F                |
| Disagreement     |T                  |None          |orth.         |1.0                |F                |


# PART2

## Tunning Intrinsic Rewards
The goal of this questions is to find which setting each of the intrinsic rewards needs to get best performance
At the end of this section, we can identify for each reward, it's best config

e.g. RND: obs_rms=True, rew_rms=True, forward_filter=True
e.g. E3B: obs_rms=False, rew_rms=True, forward_filter=False

### Q1: The impact of different Observation normalization mechanisms on final performance. (Roger)
- RL algo: PPO
- rewards: ICM, RND, RIDE, RE3, E3B
- candidates: 
  + x/255.
  + rms
- games:
  + SuperMarioBros-1-1-v3
- framework: rllte

| Name     | RND      | ICM      | Pseudocounts | RIDE   | E3B      | RE3 | NGU   | Disagreement |
|----------|----------|----------|--------------|--------|----------|-----|-------|--------------|
| Done     | True     | True     | True         | True   | True     | True| True  | True         |

### Q2: The impact of different Reward normalization mechanisms on final performance. (Roger)
- RL algo: PPO (if you use E3B, use PPO not SAC)
- rewards: ICM, RND, RIDE, RE3, E3B
- candidates: 
  + vanilla
  + rms
  + min-max
- games: 
  + SuperMarioBros-1-1-v3
- framework: rllte

| Name     | RND      | ICM      | Pseudocounts | RIDE   | E3B      | RE3 | NGU   | Disagreement |
|----------|----------|----------|--------------|--------|----------|-----|-------|--------------|
| Minmax   | True     | True     | True         | True   | True     | True| True  | True         |
| None     | True     | True     | True         | True   | True     | True| True  | True         |
| RMS      | True     | True     | True         | True   | True     |True | True  | True         |


### Q3: The co-learning dynamics of policies and intrinsic rewards: (Roger)
- The problem is that many intrinsic rewards require learning auxiliary models (e.g. inverse dynamics model, forward dynamics model, etc.) and it is not clear how to co-learn them with the policy.
- RL algo: PPO
- candidates: 
  + update_proportion = 0.25
  + update_proportion = 0.75
  + update_proportion = 1.0

- rewards: ICM, RND, RIDE, RE3, E3B
- games: 
  + SuperMarioBros-1-1-v3
- framework: rllte

| Name     | RND      | ICM      | Pseudocounts | RIDE   | E3B      | RE3 | NGU   | Disagreement |
|----------|----------|----------|--------------|--------|----------|-----|-------|--------------|
| 0.1      | True     | True     | True         | True   | True     |True |True   |True          |
| 0.5      | True     | True     | True         | True   | True     | True| True  | True         |
| 1.0      | True     | True      | True         | True   | True     |True | True  | True         |

### Q4: The impact of different weight initialization mechanisms on final performance. (Roger)
- RL algo: PPO
- rewards: ALL
- candidates: 
  + orthogonal
  + pytorch default
- games:
  + SuperMarioBros-1-1-v3
- framework: rllte

| Name     | RND      | ICM      | Pseudocounts | RIDE   | E3B      | RE3 | NGU   | Disagreement |
|----------|----------|----------|--------------|--------|----------|-----|-------|--------------|
| Orthogonal| True    | True     | True         | True   | True     |True | True  | True         |
| Default   | True    | True     | True         | True   | True     |True | True  | True         |


### Q5: Is memory required for better optimizing intrinsic rewards? (Roger)
- RL algo: PPO
- rewards: ICM, RND, RIDE, RE3, E3B
- candidates: 
  + LSTM policy
  + Vanilla policy
- games: 
  + SuperMarioBros-1-1-v3
  + SuperMarioBrosRandomLevels
- framework: rllte

| Name     | RND      | ICM      | Pseudocounts | RIDE   | E3B      | RE3 | NGU   | Disagreement |
|----------|----------|----------|--------------|--------|----------|-----|-------|--------------|
| LSTm     | True     | True     | True         | True   | True     |True |True   |True          |

## Optimizing the intrinsic rewards

The goal of this section is to learn which are the best ways to optimize the RL algos with intrinsic rewards
Starting in this section, for each reward we will use the best config found with Q1, Q2, Q3, Q4

At the end of this section we know for each algorithm, how to configure it to better optimize the intrinsic rewards

e.g. PPO+RND: Separate adv estimation
e.g. PPO+E3B: Vanilla version R=ext+int

### Q6: The impact of different integration pattern on final performance, only for on-policy setting. (Mingqi)
- RL algo: PPO, PPO+LSTM
- rewards: ALL
- candidates:
  + R = E + I
  + TwoHead
  + RE3 version: multiply the estimated advantages by the intrinsic rewards 
- games: 
  + procgen_1MazeHard - 10M steps
  + procgen_AllMazeHard - 10M steps
  + procgen_1MazeMemory - 10M steps
  + procgen_AllMazeMemory - 10M steps
  + Montezuma - 200M steps
- framework: rllte

| Name     | RND      | ICM      | Pseudocounts | RIDE   | E3B      | RE3 | NGU   | Disagreement |
|----------|----------|----------|--------------|--------|----------|-----|-------|--------------|
| 1MazeHard| False    | False    |False         | False  |False     |False|False  |False         |
|AllMazeHard| False    | False    | False        | False  | False    |False|False  |False         |
|1MazeMemory| False    | False    | False        | False  | False    |False|False  |False         |
|AllMazeMemory| False    | False    | False        | False  | False    |False|False  |False         |
|Montezuma| False    | False    | False        | False  | False    |False|False  |False         |


### Q7: The impact of ForwardRewardFilter mechanisms on final performance. (Roger)
- RL algo: PPO
- rewards: ICM, RND, RIDE, RE3, E3B
- candidates:
  + Use RewardFilter + don't cut when done=True in value estimation
  - Don't use RewardFilter + cut when done=True in value estimation
- games: 
  + Montezuma - 200M steps
- framework: rllte

| Name     | RND      | ICM      | Pseudocounts | RIDE   | E3B      | RE3 | NGU   | Disagreement |
|----------|----------|----------|--------------|--------|----------|-----|-------|--------------|
| RFF      | False    | False    |False         | False  |False     |False|False  |False         |
| No-RFF   | False    | False    | False        | False  | False    |False|False  |False         |

## Research Questions

The goal of this section is to study recent research questions of interest in the literature.

e.g. Optimizing multiple intrinsic rewards together
e.g. intrinsic rewards in contextual MDPs

### Q8: Which intrinsic rewards generalize better in Contextual MDPS? (Roger)
- rewards: ALL
- candidates: ALL rewards
- games: 
  + SuperMarioBrosRandomLevels
- framework: rllte

| Name     | RND      | ICM      | Pseudocounts | RIDE   | E3B      | RE3 | NGU   | Disagreement |
|----------|----------|----------|--------------|--------|----------|-----|-------|--------------|
| -      | False    | False    |False         | False  |False     |False|False  |False         |

### Q9: The performance of mixed intrinsic rewards. (Mingqi)
- RL algo: PPO
- rewards: ICM, RND, RIDE, RE3, E3B
- candidates:
  - global + episodic (1)
  + RND+E3B
  + ICM+E3B
  + RIDE+E3B

  - global + episodic (2)
  + RND+RE3
  + ICM+RE3
  + RIDE+RE3

  - global + global
  + RND+ICM
  + RND+RIDE
  + ICM+RIDE

- games: 
  intrinsic reward only
  
  + SuperMarioBros-1-1-v3
  + SuperMarioBrosRandomLevels
  + Montezuma
  
  
- framework: rllte

# Update log

## 31/01/2024 by Mingqi
- transfer all the reward modules from `experimental` folder to `rllte.xplore.reward`, old moduels are moved to `rllte/explore/reward/backup`
- updated the `on_policy_agent.py (Line 132-169)` to adapt to the new reward base. For the convenience of experiments, we compute the irs directly without using `if self.irs is not None`. And `R=E+I`
- updated the `.compute (Line 132-169)` function of `BaseReward`, now it requires the samples to contain all the potentially useful data:
    ``` py
    for key in ['observations', 'actions', 'rewards', 'terminateds', 'truncateds', 'infos', 'next_observations']:
        assert key in samples.keys(), f"Key {key} is not in samples."
    ```
    It is simpler to understand and we can let `.compute, .watch, .update` have same arguments.

## Next Steps

- Fix pseudo_count() function for all rewards that use pseudo_counts [DONE]
    + Change fixed memory for deques
- Implement RewardFilter as in (https://github.com/vwxyzjn/cleanrl/blob/master/cleanrl/ppo_rnd_envpool.py) [DONE]
    - When computing values and using ForwardFilter, dont cut the value estimation when done=True 
        + see Cleanrl script linked above Line 411-420
- Change `off_policy_agent.py` to adapt to the new reward class. 

## 31/01/2024 by Roger
- Added `normalize()` in the base reward class and `obs_rms` bool for all rewards (some with default to True and some to False)
    + Implemented obs normalization logic in the base reward and calls to normalize in `compute()` and `update()`
    + Added initialization of obs_norm parameters in `on_policy_agent.py (line 109-122)` based on cleanrl code

- Added `update_proportion` parameter to control how big the updates are. Necessary to answer Q6
    + Updated all `update()` functions for the rewards to use the `update_proportion` parameter

- Added SuperMario bros and checked PPO can solve it. (it works in 1M steps only)

- Added calls to `self.update()` at each `self.compute()` in rewards
- Changed intrinsic reward Encoder to Mnih encoder to process 84x84 images
- Big change to schedule to better define the study


## 01/02/2024 by Mingqi
- fixed the `pseudo_counts.py` with a reasonable episodic memory;
- fixed RIDE, NGU;
- corrected the interpretations of all the arguments and code blocks;

## 05/02/2024 by Mingqi

- add the `RewardForwardFilter`, see `base_reward.py (Line68, 88, 105-107)`;
- add the `RogerRolloutStorage` class that don't cut the rewards `(Line151-153)` when done is True, see the usage in `test_pc.py`;
- I'm drawing the workflow chart of our method.

## 09/02/2024 by Mingqi
- add the `rllte_to_cleanrl.py` that combines RLeXplore and CleanRL;
- add the `rllte_to_sbs3.py`that combines RLeXplore and SB3;
- will finish the flowchart of RLeXplorea and provide an example figure of the learning curves.